In [ ]:
import os
import math
import torch
import pandas as pd

from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)
from huggingface_hub import notebook_login
from datasets import Dataset
DATASET_PATH = "poem_final.csv"  # Xóa khoảng trắng không cần thiết
df = pd.read_csv(DATASET_PATH)  # Xóa khoảng trắng giữa `.` và `read_csv`
df
def split_content(content):
    samples = []

    poem_parts = content.split("\n\n")
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split("\n")
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)

    return samples


df["content"] = df["content"].apply(lambda x: split_content(x))
df_exploded = df.explode("content")
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=["content"])
df_exploded

c:\Users\caoth\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Unnamed: 0,title,content,source,url
0,0,NaN,"[Bạn xấu như chiếc bóng, Cứ bám riết theo anh,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,NaN,"[Nhưng họ sẽ biến mất, Khi trời phủ mây đen, T...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,NaN,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,NaN,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,NaN,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
5,2,NaN,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
6,3,NaN,"[Chơi thân không có nghĩa, Không cãi nhau bao ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
7,3,NaN,"[Vấn đề là ở chỗ, Đã chơi thân với nhau, Thì p...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
8,4,NaN,"[Có thể buồn chút ít, Một mình, không người yê...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
9,5,NaN,"[Có thể có ông bố, Không yêu con; xưa nay, Đã ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [3]:
df_exploded["content"] = df_exploded["content"].apply(lambda x: "\n".join(x))
df_exploded


,Unnamed: 0,title,content,source,url
0,0,NaN,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,NaN,Nhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTứ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,NaN,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,1,NaN,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,2,NaN,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
5,2,NaN,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
6,3,NaN,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
7,3,NaN,Vấn đề là ở chỗ\nĐã chơi thân với nhau\nThì ph...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
8,4,NaN,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
9,5,NaN,Có thể có ông bố\nKhông yêu con; xưa nay\nĐã ô...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [4]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)


In [5]:
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 26
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 3
    })
})

In [ ]:
from transformers import GPT2Tokenizer

MODEL_NAME = "danghuy1999/gpt2-viwiki"
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

MAX_SEQ_LEN = 100

# Loại bỏ NaN trước khi tokenization
df_exploded = df_exploded.dropna(subset=["content"])

def preprocess_function(rows):
    return tokenizer(
        rows["content"],
        # max_length=MAX_SEQ_LEN,
        padding="max_length",
        truncation=True
    )

# Giảm `num_proc=1` nếu dùng Windows
tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,  
    remove_columns=poem_dataset["train"].column_names, 
)


Map: 100%|██████████| 3/3 [00:00<00:00, 386.07 examples/s]


In [20]:
tokenized_poem_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 26
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3
    })
})

In [ ]:
data_collator = DataCollatorForLanguageModeling ( tokenizer = tokenizer ,
                                                 mlm = False )

In [9]:
model = GPT2LMHeadModel . from_pretrained ( MODEL_NAME )


In [ ]:
training_args = TrainingArguments(
    output_dir="gpt2_viet_poem_generation",  # Bỏ khoảng trắng dư
    save_strategy="epoch",
    learning_rate=2e-5,  
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True  # Không cần dấu phẩy cuối cùng nếu không có thêm tham số
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],  
    eval_dataset=tokenized_poem_dataset["test"],  
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Huấn luyện mô hình
trainer.train()

C:\Users\caoth\AppData\Local\Temp\ipykernel_36540\1685367710.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=40, training_loss=5.784116744995117, metrics={'train_runtime': 172.8036, 'train_samples_per_second': 1.505, 'train_steps_per_second': 0.231, 'total_flos': 13268736000000.0, 'train_loss': 5.784116744995117, 'epoch': 10.0})

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Đưa mô hình lên GPU
model.to(device)
# inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()


C:\Users\caoth\AppData\Local\Temp\ipykernel_36540\605743007.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=40, training_loss=4.607056427001953, metrics={'train_runtime': 199.7099, 'train_samples_per_second': 1.302, 'train_steps_per_second': 0.2, 'total_flos': 13268736000000.0, 'train_loss': 4.607056427001953, 'epoch': 10.0})

In [13]:
import torch
print(torch.cuda.is_available())  # True nếu GPU hoạt động


True


In [16]:
import torch

# Định nghĩa prompt đầu vào
prompt = "Học học nữa học mãi\n"

# Kiểm tra xem có thể sử dụng GPU hay không
device = "cuda" if torch.cuda.is_available() else "cpu"

# Chuyển đầu vào sang tensor
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Sinh văn bản từ mô hình GPT-2
outputs = model.generate(
    inputs,
    max_new_tokens=50,  # Số lượng token tối đa được sinh thêm
    do_sample=True,  # Mẫu ngẫu nhiên thay vì greedy decoding
    top_k=50,  # Giới hạn lựa chọn token trong top 50
    top_p=0.95,  # Chỉ chọn token có xác suất cộng dồn đến 95%
    temperature=0.3,  # Điều chỉnh độ sáng tạo (cao hơn = sáng tạo hơn)
    repetition_penalty=1.2  # Giảm lặp lại từ
)

# Giải mã đầu ra
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Hiển thị kết quả
print("\nGenerated Text:\n")
for line in results.split("\n"):
    print(line)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Text:

Học học nữa học mãi
Một số ý tưởng của người
Cho đến khi người đang sống
Ta muốn có thể
Hai ngày sau, khi trời mưa
Ngày nay, người ta đang gặp nhau
I không biết
Các ngày càng đẹp
Ngay cả ngày
Khi
